In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import re
import unicodedata

base_url = 'https://www.imdb.com'

scraping https://www.imdb.com for movie attributes: director, actors, movie critic rating, user rating, movie keywords provided by imdb, 

In [ ]:
#scraping the movie links from my imdb seen movies list

section_url = 'https://www.imdb.com/list/ls001933214/?sort=list_order,asc&st_dt=&mode=detail&page=1'
#behind section_url there is a list of over 1000 movies I've seen. the list is on my private imdb account

def next_section_url(url): #function for extracting section link for the next page
    section_url = ''
    imdb_seen = get(url, headers = {"Accept-Language": "en-US, en;q=0.5"}).text
    soup = BeautifulSoup(imdb_seen, "html.parser")
    pagination = soup.find('div', attrs={'class':'list-pagination'})
    if pagination.find('a', 'flat-button lister-page-next next-page') != None:
        section_url = base_url + pagination.find('a', 'flat-button lister-page-next next-page')['href']
    return section_url

def movie_links(url): #extracting the links for each movie's imdb page
    imdb_seen = get(url, headers = {"Accept-Language": "en-US, en;q=0.5"}).text
    soup = BeautifulSoup(imdb_seen, "html.parser")
    sub_list = soup.find('div', attrs={'class':'lister list detail sub-list'})
    list_links = [base_url + dd.a['href'] for dd in sub_list.findAll('div', attrs={'class':'lister-item mode-detail'})]
    return list_links
    

In [ ]:
movie_links [] #appending the movie links list with seen movies links:

for i in range(11): #11 is for the eleven pages in my list of movies
    movie_links = movie_links + movie_links(section_url)
    section_url = next_section_url(section_url) 
    
#saving the list of links
import pickle
pickling_on = open("seen_movies_links.pickle","wb")
pickle.dump(movie_links, pickling_on)

In [ ]:
#going to be scraping the information for each movie from the movie_links list

def get_keywords(url): #extracting the keyword list that imdb provides
    imdb_kw = get(url, headers = {"Accept-Language": "en-US, en;q=0.5"}).text
    soup = BeautifulSoup(imdb_kw, "html.parser")
    kw_base = soup.find_all('td', 'soda sodavote')
    all_kw = [kw_base[i]['data-item-keyword'] for i in range(len(kw_base))]
    return all_kw

def get_actors(soup): #extracting either the first 10 actors from the actors list, or less if the movie had less
    actor_count = len(soup.find('table', 'cast_list').find_all('tr'))-1
    if actor_count<11:
        actor_list = [soup.find('table', 'cast_list').find_all('tr')[k+1].text.split('\n\n')[2][1:-2] for k in range(actor_count)]
    else:
        actor_list = [soup.find('table', 'cast_list').find_all('tr')[k+1].text.split('\n\n')[2][1:-2] for k in range(10)]
    actors = ','.join(actor_list)
    return actors


def movie_info(movie_url):
    imdb_m = get(movie_url, headers = {"Accept-Language": "en-US, en;q=0.5"}).text #with the headers parameter I am trying to avoid movie title translations
    soup_m = BeautifulSoup(imdb_m, "html.parser")
    title_and_year = soup_m.find('title').text[:-7]
    user_rating=soup_m.find('div', 'ratingValue').text[1:4]
    critic_rating=soup_m.find('div', 'titleReviewBar').contents[1].text[3:5]
    director = soup_m.find('div', 'credit_summary_item').text[11:]
    actors = get_actors(soup_m)
    keywords_link = base_url + soup_m.find('nobr').find('a')['href']
    keywords_list = get_keywords(keywords_link)
    keywords = ','.join(keywords_list)
    joined_info = {'title':title_and_year, 'user_rating':user_rating, 'critic_rating':critic_rating, 'director':director, 'actors':actors, 'keywords':keywords}
    return joined_info

In [ ]:
movie_attr=[]

#scraping movie attributes and appending to the list:
for i in range(len(movie_links)):
    informacija = movie_info(movie_links[i])
    movie_attr.append(informacija)

movie_frame = pd.DataFrame.from_dict(movie_attr)

In [ ]:
#doing some cleaning on the gathered dataset:

movie_frame['director'] = movie_frame['director'].apply(lambda x: x.strip()) #eliminating the whitespace chars
movie_frame['director'] = movie_frame['director'].apply(lambda x: x.split('(')[0]) #leaving only the first and main director of the movie
movie_frame['director'] = movie_frame['director'].apply(lambda x: x.split('|')[0]) 

#converting numeric columns from string to integer/float
#also normalizing numbers since for one column max is 10 and another max is 100 but the information type is the same
movie_frame['critic_rating'] = pd.to_numeric(movie_frame['critic_rating'], errors='coerce')
movie_frame['user_rating'] = pd.to_numeric(movie_frame['user_rating'], errors='coerce')
movie_frame['user_rating'] = movie_frame['user_rating'].apply(lambda x: x*10)
#adding columns with values of the difference between user given rating and movie critic rating
movie_frame['rating_diff_abs'] = 0
movie_frame['rating_diff_abs'] = (movie_frame['user_rating']-movie_frame['critic_rating']).abs()
#absolute difference
movie_frame['rating_diff'] = 0
movie_frame['rating_diff'] = movie_frame['user_rating']-movie_frame['critic_rating']

#splitting title and and year and creating new column for the year that the movie was released
movie_frame['year'] = 'y'
movie_frame['year'] = movie_frame['title'].apply(lambda x: x.split('(')[1])
movie_frame['year'] = movie_frame['year'].apply(lambda x: x[0:-1])
#eliminating the year part from the title
movie_frame['title'] = movie_frame['title'].apply(lambda x: x.split('(')[0][:-1]) 


In [ ]:
#this below is a list of movies that are of preferece in regards to the whole movies list
positive_sentiment = pd.read_csv('positive_names.csv', sep='delimiter', engine='python')
true_outcome = np.array(positive_sentiment['Title'])

#create a new column in the movie data frame for the clasification of the movie. True-if the movie is of preference and False-if it is not
movie_frame['outcome'] = 0
movie_frame['outcome'] = filmu_freimas['title'].isin(true_outcome)

#save the dataframe:
movie_frame.to_csv('movie_frame.csv', index=False)

scraping the web for movie reviews from review sites: https://www.rogerebert.com https://3brothersfilm.com http://www.reelviews.net https://www.nytimes.com/reviews/movies https://brightlightsfilm.com http://www.urbancinefile.com.au http://alibi.com

In [ ]:
#functions for scraping each of the review sites if there's a link to in in the imdb page of the movie
def roger_ebert(eberts_f):
    reviews=[]
    if eberts_f is not None:
        links = [base_url + i['href'] for i in eberts_f]  
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            if window.find('h2', 'page-head') is not None: #in festival reviews this is empty, and i dont need festival reviews
                #window.find('article', {'class':'ad'}).decompose()
                raw_review = window.find('div', {'itemprop':'reviewBody'}).findAll('p')
                clean_review1 = [unicodedata.normalize("NFKD", re.sub('<[^<]+?>', '', i.text.strip())) for i in raw_review] #removing inside tags and \xa0 and .strip() removes \r\n from beginning or end 
                clean_review2 = [i.replace('\r\n', ' ').replace('\r', ' ').replace('\\', '') for i in clean_review1] #some \r\n did not vanish - now deleting the manual way
                clean_review = [i for i in clean_review2 if not i in 'Advertisement']
                reviews.append(' '.join(clean_review))
    return reviews


def brothers(brothers_f):
    reviews = []
    if brothers_f is not None:
        links = [base_url + i['href'] for i in brothers_f]  
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            raw_review = window.find_all('p')
            clean_review = [re.sub('<[^<]+?>', '', i.text) for i in raw_review[:-4]]
            reviews.append(' '.join(clean_review))
    return reviews


def reel_views(reel_f):
    reviews = []
    if reel_f is not None:
        links = [base_url + i['href'] for i in reel_f] 
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            if window.find('div', {'id':'reelContent'}) is not None:
                raw_review = window.find('div', {'id':'reelContent'}).find_all('p')
                clean_review1 = [unicodedata.normalize("NFKD", re.sub('<[^<]+?>', '', i.text.strip())) for i in raw_review] #removing \r\n
                clean_review = [i.replace('\r\n', '').replace('\\', '') for i in clean_review1] #removing manually
                reviews.append(' '.join(clean_review))
    return reviews


def ny_times(times_f):
    reviews = []
    if times_f is not None:
        links = [base_url + i['href'] for i in times_f] 
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            archive = window.find('span', 'kicker-label')
            regular_rev = window.find('p', 'css-1pdd3ka etcg8100')
            if (archive is not None) and (archive.text == 'Archives'):
                raw_review = window.find_all('p', 'story-body-text story-content')
                clean_review = [i.text.strip() for i in raw_review[:-1]]
                #clean_review = [i.replace('\\', '') for i in clean_review1]
                reviews.append(' '.join(clean_review))
            elif (regular_rev is not None) and (regular_rev.text == 'Movie Review'):
                raw_review = window.find('section', {'name':'articleBody'}).find_all('p', 'css-1ygdjhk e2kc3sl0')
                clean_review = [i.text.strip() for i in raw_review[:-1]]
                #clean_review = [i.replace('\\', '') for i in clean_review1]
                reviews.append(' '.join(clean_review))      
    return reviews

                
def bright_lights(bl_f):
    reviews = []
    if bl_f is not None:
        links = [base_url + i['href'] for i in bl_f]  
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            raw_review = window.find('div', 'text').text.strip()
            clean_review = raw_review.replace('\r\n', '').replace('\n', '')
            reviews.append(clean_review)
    return reviews


def alibi(alibi_f):
    reviews = []
    if alibi_f is not None:
        links = [base_url + i['href'] for i in alibi_f] 
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            clean_review = window.find('div', {'itemprop':'reviewBody'}).text
            reviews.append(clean_review)
    return reviews     
    
    
def cinefile(cinefile_f):
    reviews = []
    if cinefile_f is not None:
        links = [base_url + i['href'] for i in cinefile_f] 
        for i in range(len(links)):
            window = BeautifulSoup(get(links[i]).text, "html.parser")
            b_tags = window.find('font', {'class':'articleBody'}).find_all('b') #eliminate b tags
            for i in b_tags:
                i.decompose()
            raw_review = window.find('font', {'class':'articleBody'}).text.strip()
            clean_review = raw_review.replace('\r', '').replace('\n', '')
            reviews.append(clean_review)
    return reviews

In [ ]:
#function for gathering the movie title and all available reviews
def the_reviews(url):
    imdb_m = get(url, headers = {"Accept-Language": "en-US, en;q=0.5"}).text 
    soup_m = BeautifulSoup(imdb_m, "html.parser")
    title = soup_m.find('title').text[:-7] #movie title
    metas = soup_m.find('div','titleReviewBarItem titleReviewbarItemBorder').find_all('a')[1]['href']
    if metas is not None:
        metas_soup = BeautifulSoup(get(url+metas).text, "html.parser")
        eberts_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['Roger Ebert', 'Rogerebert.com']])
        brothers_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['3 Brothers']])
        reel_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['ReelViews']])
        times_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['New York Times']])
        bl_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['Bright Lights Film Journal']])
        alibi_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['Alibi.com']])
        cinefile_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['Urban Cinefile']])
        flick_f = metas_soup.find_all('a', text = [re.compile(p) for p in ['FlickFilosopher.com']])
        
        lights_reviews = bright_lights(bl_f)
        alibi_reviews = alibi(alibi_f)
        cinefile_reviews = cinefile(cinefile_f)
        ebert_reviews = roger_ebert(eberts_f)
        brothers_reviews = brothers(brothers_f)
        reel_reviews = reel_views(reel_f)
        times_reviews = ny_times(times_f)

    info = {'title':title, 'ebert':ebert_reviews, '3brothers':brothers_reviews, 'reel':reel_reviews, 'nytimes':times_reviews, 'bright_lights':lights_reviews, 'alibi':alibi_reviews, 'cinefile':cinefile_reviews}
    return info


#filling an empty list with movie title and all availabe reviews for that particular movie:
review_frame=[]
for i in movie_links:
    review_frame.append(the_reviews(i))

since one movie can have multiple critic reviews, below is the part where a list consisting of reviews and outcome solely will be created.
this way the 1007 movie sample will be expanded 

In [ ]:
review_and_outcome = []

for i in review_frame:
    if len(i['3brothers'])>0:
        skaicius = len(i['3brothers']) 
        for k in range(skaicius):
                review_and_outcome.append([i['3brothers'][k], i['outcome']])
    if len(i['ebert'])>0:
        skaicius = len(i['ebert']) 
        for k in range(skaicius):
            review_and_outcome.append([i['ebert'][k], i['outcome']])
    if len(i['nytimes'])>0:
        skaicius = len(i['nytimes']) 
        for k in range(skaicius):
            review_and_outcome.append([i['nytimes'][k], i['outcome']])
    if len(i['reel'])>0:
        skaicius = len(i['reel']) 
        for k in range(skaicius):
            review_and_outcome.append([i['reel'][k], i['outcome']])
    if len(i['bright_lights'])>0:
        skaicius = len(i['bright_lights']) 
        for k in range(skaicius):
            review_and_outcome.append([i['bright_lights'][k], i['outcome']])
    if len(i['alibi'])>0:
        skaicius = len(i['alibi']) 
        for k in range(skaicius):
            review_and_outcome.append([i['alibi'][k], i['outcome']])
    if len(i['cinefile'])>0:
        skaicius = len(i['cinefile']) 
        for k in range(skaicius):
            review_and_outcome.append([i['cinefile'][k], i['outcome']])
        
df = pd.DataFrame(review_and_outcome, columns=['review', 'outcome'])
df.to_csv('review_and_outcome.csv', index=False)